In [1]:
import pickle
import torchvision
import torch
from torchvision.models.detection.ssd import SSDClassificationHead
from torchvision.models.detection import _utils
from torchvision.models.detection import SSD300_VGG16_Weights
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
from PIL import Image
from torchvision import tv_tensors 
from tqdm.auto import tqdm
import albumentations as A
from torch.optim.lr_scheduler import StepLR
from albumentations.pytorch import ToTensorV2

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
!pip install pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 15.4 MB/s eta 0:00:00


In [3]:
!pip install faster-coco-eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.9/470.9 kB 14.3 MB/s eta 0:00:00


In [4]:
!pip install torchmetrics[detection]

In [5]:
!pip install -U albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.9/227.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.1/605.1 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.7/291.7 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.17
    Uninstalling albucore-0.0.17:
      Successfully uninstalled albucore-0.0.17
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.17
    Uninstalling albumentations-1.4.17:
      Successfully uninstalled albumentations-1.4.17


In [6]:
def create_model(num_classes=91, size=300):
    # Load the Torchvision pretrained model.
    model = torchvision.models.detection.ssd300_vgg16(
        weights=SSD300_VGG16_Weights.COCO_V1
    )
    # Retrieve the list of input channels. 
    in_channels = _utils.retrieve_out_channels(model.backbone, (size, size))
    # List containing number of anchors based on aspect ratios.
    num_anchors = model.anchor_generator.num_anchors_per_location()
    # The classification head.
    model.head.classification_head = SSDClassificationHead(
        in_channels=in_channels,
        num_anchors=num_anchors,
        num_classes=num_classes,
    )
    # Image size for transforms.
    model.transform.min_size = (size,)
    model.transform.max_size = size
    return model

In [7]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [8]:
epsilon = 1e-6
def transform_box(box, height, width):
    
    x_center, y_center, box_width, box_height = box
    # Giới hạn biên cho các bounding box 
    x_min = max((x_center - box_width / 2) * width, 0 + epsilon)
    x_max = min((x_center + box_width / 2) * width, width - epsilon)
    y_min = max((y_center - box_height / 2) * height, 0 + epsilon)
    y_max = min((y_center + box_height / 2) * height, height - epsilon)

    return [x_min, y_min, x_max, y_max]
    

def read_filetxt(file_txt):
    results = {'labels': [], 'boxes': []}
    with open(file_txt, "r", newline='') as file:
        for line in file.readlines():
            values = line.strip().split()
            if int(values[0]) >= 4:
                values[0] = int(values[0]) - 3
            else:
                values[0] = int(values[0]) + 1
            
            results['labels'].append(int(values[0]))
            results['boxes'].append([float(value) for value in values[1: 5]])

    return results

def get_file_path(folder):
    data = []
    sub_folders = list(sorted(os.listdir(folder)))
    sub_folder_paths = [os.path.join(folder, sub_folder) for sub_folder in sub_folders]
    # Kiểm tra tập train hay test
    try:
        contents = os.listdir(sub_folder_paths[0])
        if contents:
            # Duyêt qua tập daytime và nighttime
            for sub_folder_path in sub_folder_paths:
                data_paths = list(sorted(os.listdir(sub_folder_path)))
                data_path_file = [os.path.join(sub_folder_path, data_path) for data_path in data_paths]
                data.extend(data_path_file)

            return {'folder': 'train', 'file_train': data}
    except NotADirectoryError:
        return {'folder': 'test', 'file_test': sub_folder_paths}     

class TrafficVehicle(Dataset):
    def __init__(self, folder: str, transforms=None, 
                 transform_box_type: str="center"):
        """ 
            transform_box sẽ chuyển tọa độ bounding box sang để phù hợp với các model, có hai kiểu để chuyển đổi 
            là "center" hoặc "corner", mặc định là "center". Nếu transform_type="center" thì mặc định các model sử dụng bounding box 
            theo tọa độ tâm và kích thước. Ngược lại, transform_type="corner" các model sẽ sử dụng bounding box theo tọa độ góc.

            Ví dụ: trong bài toán object detection các tọa độ bounding box thường chia thành hai kiểu:
                - (x_min, y_min, x_max, y_max)
                - (x_center, y_center, width, height)
            
            Lưu ý: Do trong quá trình gán nhãn theo định dạng YOLO (x_center, y_center, width, height) nên không cần transform_box nữa.
        """
        self.data = get_file_path(folder)
        self.transforms = transforms
        self.transform_box_type = transform_box_type

        if self.data['folder'] == 'train':
            self.image, self.txt = [], []
            for train_path in self.data['file_train']:
                train_path_split = train_path.split('.')
                # Kiểm tra là txt hay là image
                if train_path_split[-1] == 'txt':
                    self.txt.append(train_path)
                else:
                    self.image.append(train_path)
        else:
            self.image = [image_path for image_path in self.data['file_test']]
        self.class_name = {1: 'motocycle', 2: 'car', 3: 'coach', 4: 'container truck'}
    def load_image(self, index):
        img = Image.open(self.image[index])
        img = np.array(img)
        return img
    def __len__(self):
        return len(self.image)
    def __getitem__(self, index: int):
        img = self.load_image(index)
        file_path = os.path.basename(self.image[index])
        if self.data['folder'] == 'train': 
            target = read_filetxt(self.txt[index])
    
            # kiểm tra transform_box có phải là center hoặc corner không
            if self.transform_box_type not in ['center', 'corner']:
                raise ValueError("transform_box_type phải là 'center' hoặc 'corner'!")
            
            # transform_box bounding box 
            if self.transform_box_type == "corner":
                height, width = img.shape[:2]
                target['boxes'] = [transform_box(box, height, width) for box in target['boxes']]
    
            # Áp dụng transforms cho image và box
            if self.transforms is not None:
                transformed = self.transforms(image=img, bboxes=target['boxes'], class_labels=target['labels'])
                img = transformed['image']
                target['boxes'] = transformed['bboxes']
                target['labels'] = transformed['class_labels']
    
            target['boxes'] = torch.tensor(target['boxes'], dtype=torch.float32)
            target['labels'] = torch.tensor(target['labels'], dtype=torch.int64)
            return img, target
        else:
            if self.transforms is not None:
                transformed = self.transforms(image=img)
                img = transformed['image']
            return img, file_path

In [9]:
def train(train_data_loader, model):
    print('Training')
    model.train()

     # initialize tqdm progress bar
    prog_bar = tqdm(train_data_loader, total=len(train_data_loader))
    
    for i, data in enumerate(prog_bar):
        optimizer.zero_grad()
        images, targets = data
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        train_loss_hist.send(loss_value)

        losses.backward()
        optimizer.step()
    
        # update the loss value beside the progress bar for each iteration
        prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")
    return loss_value

In [10]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision
def validate(valid_data_loader, model):
    print('Validating')
    model.eval()
    
    # Initialize tqdm progress bar.
    prog_bar = tqdm(valid_data_loader, total=len(valid_data_loader))
    target = []
    preds = []
    for i, data in enumerate(prog_bar):
        images, targets = data
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
        
        with torch.no_grad():
            outputs = model(images, targets)

        # For mAP calculation using Torchmetrics.
        #####################################
        for i in range(len(images)):
            true_dict = dict()
            preds_dict = dict()
            true_dict['boxes'] = targets[i]['boxes'].detach().cpu()
            true_dict['labels'] = targets[i]['labels'].detach().cpu()
            preds_dict['boxes'] = outputs[i]['boxes'].detach().cpu()
            preds_dict['scores'] = outputs[i]['scores'].detach().cpu()
            preds_dict['labels'] = outputs[i]['labels'].detach().cpu()
            preds.append(preds_dict)
            target.append(true_dict)
        #####################################

    metric = MeanAveragePrecision(class_metrics=True)
    metric.update(preds, target)
    metric_summary = metric.compute()
    return metric_summary

In [11]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0
        
    def send(self, value):
        self.current_total += value
        self.iterations += 1
    
    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations
    
    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [12]:
train_transform = A.Compose([
        A.Resize(640, 640),
        A.Blur(blur_limit=3, p=0.1),
        A.MotionBlur(blur_limit=3, p=0.1),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.ToGray(p=0.3),
        A.RandomBrightnessContrast(p=0.3),
        A.ColorJitter(p=0.3),
        A.RandomGamma(p=0.3),
        A.ToFloat(max_value=255.0),
        ToTensorV2(p=1.0),  # Convert to tensor
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

val_transform = A.Compose([
        A.Resize(640, 640),
        A.ToFloat(max_value=255.0),
        ToTensorV2(p=1.0),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

train_data_path = '/kaggle/input/traffic-vehicle-detection/data/data/train'
train_dataset = TrafficVehicle(train_data_path, transforms=train_transform, transform_box_type="corner")
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
# images, targets = next(iter(train_loader))
# for image,target in zip(images,targets):
#     print(image)
#     print(target)
#     break
val_data_path = '/kaggle/input/traffic-vehicle-detection/data/data/val'
val_dataset = TrafficVehicle(val_data_path, transforms=val_transform, transform_box_type="corner")
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: tuple(zip(*x))) 

model = create_model(num_classes=5, size=640)
model = model.to(DEVICE)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, nesterov=True)
scheduler = StepLR(
    optimizer=optimizer, step_size=10, gamma=0.1, verbose=True
)
num_epoch = 40
train_loss_hist = Averager()

# To store training loss and mAP values.
train_loss_list = []
map_list = []
map_per_class_list = []
best_valid_map = 0
patience = 4
patience_counter = 0

for epoch in range(num_epoch):
    print(f"\nEPOCH {epoch+1} / {num_epoch}")
    train_loss_hist.reset()
    train_loss = train(train_loader, model)
    val_metric = validate(val_loader, model)
    print(f"Epoch #{epoch+1} train loss: {train_loss_hist.value:.3f}")   
    print(f"Epoch #{epoch+1} mAP: {val_metric['map']}")

    train_loss_list.append(train_loss)
    map_list.append(val_metric['map'])
    map_per_class_list.append(val_metric['map_per_class'])
    current_valid_map = val_metric['map']
    
    if current_valid_map > best_valid_map:
        best_valid_map = current_valid_map
        patience_counter = 0
        print(f"\nBEST VALIDATION mAP: {best_valid_map}")
        print(f"\nSAVING BEST MODEL FOR EPOCH: {epoch+1}\n")
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            }, "best_model.pth")
    else:
        patience_counter += 1
        print(f"No improvement in map score for {patience_counter} epochs.")
    if patience_counter >= patience:
        print("Early stopping triggered. No improvement in map score.")
        break
    scheduler.step()

Downloading: "https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth" to /root/.cache/torch/hub/checkpoints/ssd300_vgg16_coco-b556d3b4.pth
100%|██████████| 136M/136M [00:05<00:00, 25.8MB/s]



EPOCH 1 / 40
Training


  0%|          | 0/296 [00:00<?, ?it/s]

Validating


  0%|          | 0/65 [00:00<?, ?it/s]

...

Epoch #39 train loss: 1.823
Epoch #39 mAP: 0.43845078349113464

BEST VALIDATION mAP: 0.43845078349113464

SAVING BEST MODEL FOR EPOCH: 39


EPOCH 40 / 40
Training


  0%|          | 0/296 [00:00<?, ?it/s]

Validating


  0%|          | 0/65 [00:00<?, ?it/s]

Epoch #40 train loss: 1.817
Epoch #40 mAP: 0.4384986162185669

BEST VALIDATION mAP: 0.4384986162185669

SAVING BEST MODEL FOR EPOCH: 40



In [13]:
train_loss_list

[3.1964900493621826,
 2.9643545150756836,
 2.6642422676086426,
 2.563961982727051,
 2.443805694580078,
 2.2543885707855225,
 2.2247793674468994,
 2.1642425060272217,
 2.018636703491211,
 1.8832051753997803,
 1.926539659500122,
 1.9637246131896973,
 1.9504315853118896,
 1.8659582138061523,
 1.8627519607543945,
 1.8994925022125244,
 1.903616189956665,
 1.797156572341919,
 1.900951862335205,
 1.847141981124878,
 1.7862330675125122,
 1.7566871643066406,
 1.7504079341888428,
 1.7497193813323975,
 1.890145182609558,
 1.789435863494873,
 1.8134346008300781,
 1.724400520324707,
 1.7465896606445312,
 1.7528879642486572,
 1.783158302307129,
 1.8171417713165283,
 1.7740693092346191,
 1.8348815441131592,
 1.761590838432312,
 1.855651617050171,
 1.7031300067901611,
 1.7409930229187012,
 1.7752480506896973,
 1.7141327857971191]

In [14]:
map_list

[tensor(0.1551),
 tensor(0.2414),
 tensor(0.2839),
 tensor(0.3035),
 tensor(0.3185),
 tensor(0.3330),
 tensor(0.3440),
 tensor(0.3557),
 tensor(0.3612),
 tensor(0.3666),
 tensor(0.3995),
 tensor(0.4089),
 tensor(0.4147),
 tensor(0.4168),
 tensor(0.4198),
 tensor(0.4215),
 tensor(0.4237),
 tensor(0.4242),
 tensor(0.4258),
 tensor(0.4261),
 tensor(0.4338),
 tensor(0.4356),
 tensor(0.4359),
 tensor(0.4362),
 tensor(0.4367),
 tensor(0.4370),
 tensor(0.4368),
 tensor(0.4370),
 tensor(0.4376),
 tensor(0.4377),
 tensor(0.4377),
 tensor(0.4379),
 tensor(0.4381),
 tensor(0.4382),
 tensor(0.4383),
 tensor(0.4382),
 tensor(0.4383),
 tensor(0.4383),
 tensor(0.4385),
 tensor(0.4385)]

In [15]:
map_per_class_list

[tensor([0.1670, 0.2201, 0.0853, 0.1481]),
 tensor([0.1910, 0.3005, 0.1907, 0.2835]),
 tensor([0.2094, 0.3390, 0.2460, 0.3413]),
 tensor([0.2127, 0.3604, 0.2844, 0.3563]),
 tensor([0.2194, 0.3758, 0.3101, 0.3688]),
 tensor([0.2246, 0.3897, 0.3381, 0.3797]),
 tensor([0.2271, 0.4016, 0.3591, 0.3882]),
 tensor([0.2314, 0.4104, 0.3851, 0.3960]),
 tensor([0.2328, 0.4102, 0.3964, 0.4055]),
 tensor([0.2343, 0.4076, 0.4153, 0.4093]),
 tensor([0.2517, 0.4375, 0.4806, 0.4283]),
 tensor([0.2574, 0.4447, 0.4966, 0.4368]),
 tensor([0.2608, 0.4470, 0.5070, 0.4441]),
 tensor([0.2617, 0.4495, 0.5095, 0.4465]),
 tensor([0.2637, 0.4514, 0.5139, 0.4502]),
 tensor([0.2642, 0.4521, 0.5183, 0.4515]),
 tensor([0.2659, 0.4529, 0.5225, 0.4535]),
 tensor([0.2663, 0.4536, 0.5221, 0.4548]),
 tensor([0.2668, 0.4542, 0.5261, 0.4561]),
 tensor([0.2675, 0.4548, 0.5262, 0.4557]),
 tensor([0.2751, 0.4604, 0.5391, 0.4606]),
 tensor([0.2781, 0.4615, 0.5405, 0.4624]),
 tensor([0.2793, 0.4614, 0.5400, 0.4629]),
 tensor([0.

In [16]:
with open('score.pkl', 'wb') as f:
    pickle.dump([train_loss_list, map_list, map_list], f)

**Test**

In [17]:
test_transform = A.Compose([
        A.Resize(640, 640),
        A.ToFloat(max_value=255.0),
        ToTensorV2(p=1.0),
    ])
test_data_path = '/kaggle/input/traffic-vehicle-detection/public test/public test'
test_dataset = TrafficVehicle(test_data_path, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [18]:
# model = create_model(num_classes=5)
# model = model.to(DEVICE)
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, nesterov=True)
# scheduler = StepLR(
#     optimizer=optimizer, step_size=10, gamma=0.1, verbose=True
# )

In [19]:
# checkpoint = torch.load('/kaggle/input/traffic-vehicle-detection/best_model.pth')
# model.load_state_dict(checkpoint['model_state_dict'])

In [20]:
model.eval()
output_lines = []
with torch.inference_mode():
    print('Testing')

     # initialize tqdm progress bar
    prog_bar = tqdm(test_loader, total=len(test_loader))
    for i, data in enumerate(prog_bar): 
        images, file_names = data
        images = list(image.to(DEVICE) for image in images)
        outputs = model(images)
        outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]
        for i, (output, file_name) in enumerate(zip(outputs, file_names)):
            boxes = output['boxes'].data.numpy()
            labels = output['labels'].data.numpy()
            scores = output['scores'].data.numpy()
            boxes = boxes[scores >= 0.25]
            labels = labels[scores >= 0.25] - 1
            scores = scores[scores >= 0.25]
            for box, label, score in zip(boxes, labels, scores):
                xmin = (box[0] / 640) 
                ymin = (box[1] / 640) 
                xmax = (box[2] / 640) 
                ymax = (box[3] / 640) 
                x_center = (xmin + xmax) / 2.0
                y_center = (ymin + ymax) / 2.0
                width = xmax - xmin
                height = ymax - ymin
                line = f"{file_name} {label.item()} {x_center.item()} {y_center.item()} {width.item()} {height.item()} {score.item()}"
                output_lines.append(line)

# Save to a file
with open("predict.txt", "w") as f:
    f.write("\n".join(output_lines))
                

Testing


  0%|          | 0/65 [00:00<?, ?it/s]